In [6]:
from medvqa.utils.files import load_jsonl, load_pickle

In [2]:
integrated_fact_metadata = load_jsonl("/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(578718,58628868).jsonl")

In [4]:
len(integrated_fact_metadata)

578718

In [81]:
integrated_fact_metadata[1]

{'fact': 'worsening osteoporosis in a debilitated and wheelchair-bound patient',
 'metadata': {'anatomical location': '',
  'detailed observation': 'worsening osteoporosis',
  'short observation': 'osteoporosis',
  'category': 'disease',
  'health status': 'abnormal',
  'prev_study_comparison?': 'no',
  'comparison status': ''},
 'extraction_method': 'gpt-3.5-turbo-0613'}

In [7]:
anat_loc_embeddings = load_pickle('/mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/anatomical_location_embeddings(BiomedVLP-CXR-BERT-specialized,28347,779891).pkl')

In [34]:
from nltk.tokenize import word_tokenize

word2anatlocs = {}
for x in anat_loc_embeddings['anatomical locations']:
    for word in word_tokenize(x):
        try:
            word2anatlocs[word].append(x)
        except KeyError:
            word2anatlocs[word] = [x]

In [80]:
word = 'line'
# word = 'gastroesophageal'
len(word2anatlocs[word])

318

In [52]:
import random
random.sample(word2anatlocs[word], min(20,len(word2anatlocs[word])))

['upper esophageal sphincter']

In [63]:
reload(medvqa.utils.math)

<module 'medvqa.utils.math' from '/home/pamessina/medvqa/medvqa/utils/math.py'>

In [64]:
from medvqa.utils.math import rank_vectors_by_consine_similarity, rank_vectors_by_dot_product
import random

# query_idx = random.choice(range(anat_loc_embeddings['embeddings'].shape[0]))
# query_idx = anat_loc_embeddings['anatomical locations'].index('upper part of non-distended stomach')
query_idx = anat_loc_embeddings['anatomical locations'].index('region of the cardioesophageal junction')
# query_idx = anat_loc_embeddings['anatomical locations'].index('right lung')

query_embedding = anat_loc_embeddings['embeddings'][query_idx]
# ranked_indices = rank_vectors_by_consine_similarity(anat_loc_embeddings['embeddings'], query_embedding)
ranked_indices = rank_vectors_by_dot_product(anat_loc_embeddings['embeddings'], query_embedding)

print(query_idx)
print(anat_loc_embeddings['anatomical locations'][query_idx])
print('----')

for j, i in enumerate(ranked_indices):
    print(anat_loc_embeddings['anatomical locations'][i])
    if j > 30:
        break

6065
region of the cardioesophageal junction
----
region of the cardioesophageal junction
cardioesophageal junction
posterior aspect of the costophrenic sinus
lower thoracic esophagus above the thoracoabdominal junction
aortopulmonic level
adjacent to the descending thoracic aorta above the diaphragm
posterior aspect of the costophrenic sulcus
lateral inferior costophrenic angles
posterior aspect of the costophrenic sinuses
inferior thoracic
inferior most aspects of the costophrenic sulci
lateral aspect of the costophrenic sinus
thoracolumbar junction (T12-L1)
posterior costophrenic angle
posterior CP angles
abdominal fat pad
posterior aspect of the sinuses
posterior to the cardiac silhouette
lower thoracic spine to the right
posterior costophrenic sulci
posterior cardiophrenic
deep posterior costophrenic sulci
inferior most aspects of the costophrenic angles
very inferior posterior costophrenic angles
non posterior costophrenic angles
posterior CP angle
posterior to the heart
inferior

In [10]:
anatloc2embedding = {x:y for x,y in zip(anat_loc_embeddings['anatomical locations'], anat_loc_embeddings['embeddings'])}

In [15]:
from importlib import reload
import medvqa

In [73]:
reload(medvqa.datasets.fact_ranking_rules.anatomical_location)

<module 'medvqa.datasets.fact_ranking_rules.anatomical_location' from '/home/pamessina/medvqa/medvqa/datasets/fact_ranking_rules/anatomical_location.py'>

In [74]:
from medvqa.datasets.fact_ranking_rules.anatomical_location import AnatomicalLocationTripletRanker

In [75]:
ranker = AnatomicalLocationTripletRanker(anatloc2embedding)

In [77]:
ranker.run_tests('/home/pamessina/medvqa/medvqa/datasets/fact_ranking_rules/tests/anatomical_location_triplets.json')

Unsure: unsure prediction for
	query=region of the cardioesophageal junction
	fact1=bottom of the image at the region of the cardioesophageal junction
	fact2=posterior aspect of the costophrenic sinus
	sim1=0.5641763210296631
	sim2=0.9537634253501892
	levd1=27
	levd2=31
	label=1
	pred=0
Unsure: unsure prediction for
	query=region of the cardioesophageal junction
	fact1=bottom of the image at the region of the cardioesophageal junction
	fact2=posterior aspect of the costophrenic sulcus
	sim1=0.5641763210296631
	sim2=0.9423143863677979
	levd1=27
	levd2=30
	label=1
	pred=0
Unsure: unsure prediction for
	query=region of the cardioesophageal junction
	fact1=bottom of the image at the region of the cardioesophageal junction
	fact2=lateral inferior costophrenic angles
	sim1=0.5641763210296631
	sim2=0.9414252638816833
	levd1=27
	levd2=31
	label=1
	pred=0
Unsure: unsure prediction for
	query=region of the cardioesophageal junction
	fact1=bottom of the image at the region of the cardioesophageal

In [19]:
ranker.rank('L lung', 'left lung', 'right lung')

0.9801390171051025 0.3071393072605133


1

In [20]:
ranker.rank('L lung', 'right lung', 'left lung')

0.3071393072605133 0.9801390171051025


-1